# 01-femsainfancias_missingcities_osmnxnetwork

This notebook runs the missing cities of project femsainfancias, creates a Network (G,nodes,edges) for a bbox around the city and saves it to db.

## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

In [3]:
##### WARNING ##### WARNING ##### WARNING ##### WARNING ##### WARNING #####

# save output to database?
save = True

##### WARNING ##### WARNING ##### WARNING ##### WARNING ##### WARNING #####

## Load cities

In [2]:
# gdf_mun
metro_schema = 'projects_research'
metro_table = 'femsainfancias_missingcities_metrogdf2020'

query = f"SELECT * FROM {metro_schema}.{metro_table}"
metro_gdf = aup.gdf_from_query(query, geometry_col='geometry')
metro_gdf = metro_gdf.set_crs("EPSG:4326")

city_list = list(metro_gdf.city.unique())

# Show
print(metro_gdf.shape)
print(len(city_list))
print(city_list)
metro_gdf.head(1)

(4, 6)
2
['Zihuatanejo', 'San Cristobal']


,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,city,geometry
0,12038,12,038,Zihuatanejo de Azueta,Zihuatanejo,"MULTIPOLYGON (((-101.65842 17.67894, -101.6580..."


## Run function create_osmnx_network for each city and save

In [4]:
save_schema = 'projects_research'
nodes_save_table = 'femsainfancia_missingcities_nodes'
edges_save_table = 'femsainfancia_missingcities_edges'

In [9]:
k = len(city_list)

# prevent cities being analyzed several times in case of a crash
nodes_processed_city_list = []
edges_processed_city_list = []

try:
    query = f"SELECT city FROM {save_schema}.{nodes_save_table}"
    nodes_processed = aup.df_from_query(query)
    nodes_processed_city_list = list(nodes_processed.city.unique())
    
    query = f"SELECT city FROM {save_schema}.{edges_save_table}"
    edges_processed = aup.df_from_query(query)
    edges_processed_city_list = list(edges_processed.city.unique())

except:
    pass

# PRINT PROGRESS
missing_cities_nodes = []
for city in city_list:
    if city not in nodes_processed_city_list:
        missing_cities_nodes.append(city)

i = len(nodes_processed_city_list)
print(f'Already processed nodes for ({i}/{k}) cities.')
print(f'Missing nodes procesing for cities: {missing_cities_nodes}')

missing_cities_edges = []
for city in city_list:
    if city not in edges_processed_city_list:
        missing_cities_edges.append(city)

j = len(nodes_processed_city_list)
print(f'Already processed nodes for ({i}/{k}) cities.')
print(f'Missing nodes procesing for cities: {missing_cities_nodes}')    

# SKIP FAILED CITIES
skip_city_list = []

# RUN ALL OTHER CITIES
for city in city_list:
    if city not in skip_city_list:
        if (city not in nodes_processed_city_list) and (city not in edges_processed_city_list):
            print("--"*40)
            i = i + 1
            print(f"Starting city {i}/{k}: {city}")
        
            # Load area of interest
            city_gdf = metro_gdf.loc[metro_gdf.city == city]
            aoi = city_gdf.dissolve()
            
            # Create Network
            G,nodes,edges = aup.create_osmnx_network(aoi, how='from_bbox', network_type='all_private')

            nodes['city'] = city
            edges['city'] = city
        
            # Save Network
            if save:

                # Resets index (allows delete duplicates in DBeaver using u, v, osmid)
                nodes = nodes.reset_index()
                edges = edges.reset_index()
                
                print(f"Uploading nodes for {city}")
                aup.gdf_to_db_slow(nodes, nodes_save_table, save_schema, if_exists='append')
                print(f"Uploading edges for {city}")
                aup.gdf_to_db_slow(edges, edges_save_table, save_schema, if_exists='append')

        else:
            print("--"*40)
            print(f"{city} already processed. ({i}/{k})")
            
    else:
        print("--"*40)
        print(f"SKIPPED {city}")

Already processed nodes for (0/2) cities.
Missing nodes procesing for cities: ['Zihuatanejo', 'San Cristobal']
Already processed nodes for (0/2) cities.
Missing nodes procesing for cities: ['Zihuatanejo', 'San Cristobal']
--------------------------------------------------------------------------------
Starting city 1/2: Zihuatanejo
Extracted min and max coordinates from the municipality. Polygon N:18.06141, S:17.55477, E-101.20053, W-101.71494.
Created OSMnx graph from bounding box.
Converted OSMnx graph to 10802 nodes and 26727 edges GeoDataFrame.
Added column width for edges.
Filtered columns.
Column: osmid in nodes gdf, has a list in it, the column data was converted to string.
Column: name in nodes gdf, has a list in it, the column data was converted to string.
Column: highway in nodes gdf, has a list in it, the column data was converted to string.
Uploading nodes for Zihuatanejo
Uploading edges for Zihuatanejo
-----------------------------------------------------------------------